In [ ]:
__author__ = "Jithin Pradeep"
__copyright__ = "Copyright (C) 2018 Jithin Pradeep"
__license__ = "MIT License"
__version__ = "1.0"

In [ ]:
def label2int(ch):
    asciiVal = ord(ch)
    if(asciiVal<=57): #0-9
        asciiVal-=48
    elif(asciiVal<=90): #A-Z
        asciiVal-=55
    else: #a-z
        asciiVal-=61
    return asciiVal
    
def int2label(i):
    if(i<=9): #0-9
        i+=48
    elif(i<=35): #A-Z
        i+=55
    else: #a-z
        i+=61
    return chr(i)

In [ ]:
import os
import glob
import pandas as pd
import math

import numpy as np
from scipy.misc import imread, imsave, imresize
from natsort import natsorted


# Path of data files
path = "../data"

# Input image dimensions
img_rows, img_cols = 32,32 

# Keep or not the initial image aspect ratio
keepRatio = False

# Suffix of the created directories and files
suffix = "Preproc_" + str(img_rows) + "_"+ str(img_cols) + ("_kr" if keepRatio else "")

# Create the directories if needed
if not os.path.exists(os.getcwd()+"/data/train"+suffix ):
    os.makedirs(os.getcwd()+"/data/train"+suffix)
if not os.path.exists( os.getcwd()+"/data/test"+suffix ):
    os.makedirs(os.getcwd()+"/data/test"+suffix)
    
    
### Images preprocessing ###

for setType in ["train", "test"]:
    # We have to make sure files are sorted according to labels, even if they don't have trailing zeros
    files = natsorted(glob.glob(os.getcwd()+"/data/"+setType+"/*"))
    #print(files) 
    data = np.zeros((len(files), img_rows, img_cols)) #will add the channel dimension later
    
    for i, filepath in enumerate(files):
        #print(filepath)
        image = imread(filepath, True) #True: flatten to grayscale
        if keepRatio:
            # Find the largest dimension (height or width)
            maxSize = max(image.shape[0], image.shape[1])
            
            # Size of the resized image, keeping aspect ratio
            imageWidth = math.floor(img_rows*image.shape[0]/maxSize)
            imageHeigh = math.floor(img_cols*image.shape[1]/maxSize)
            
            # Compute deltas to center image (should be 0 for the largest dimension)
            dRows = (img_rows-imageWidth)//2
            dCols = (img_cols-imageHeigh)//2
                        
            imageResized = np.zeros((img_rows, img_cols))
            imageResized[dRows:dRows+imageWidth, dCols:dCols+imageHeigh] = imresize(image, (imageWidth, imageHeigh))
            
            # Fill the empty image with the median value of the border pixels
            # This value should be close to the background color
            val = np.median(np.append(imageResized[dRows,:],
                                      (imageResized[dRows+imageWidth-1,:],
                                      imageResized[:,dCols],
                                      imageResized[:,dCols+imageHeigh-1])))
                                      
            # If rows were left blank
            if(dRows>0):
                imageResized[0:dRows,:].fill(val)
                imageResized[dRows+imageWidth:,:].fill(val)
                
            # If columns were left blank
            if(dCols>0):
                imageResized[:,0:dCols].fill(val)
                imageResized[:,dCols+imageHeigh:].fill(val)
        else:
            imageResized = imresize(image, (img_rows, img_cols))
        
        # Add the resized image to the dataset
        data[i] = imageResized
        
        #Save image (mostly for visualization)
        #filename = filepath.split("\\")[-1]
        #print(filname)
        #filenameDotSplit = filename.split(".")
        #print(filenameDotSplit)
        #newFilename = str((filenameDotSplit[0])).zfill(5) + "." + filenameDotSplit[-1].lower()  #Add trailing zeros
        #print(str((filenameDotSplit[0])))
        #print(newFilename)
        #newName = "/".join(filepath.split("\\")[:-1] ) +suffix+ "/" + newFilename
        # print(newName)
        # imsave(newName, imageResized)
        
    # Add channel/filter dimension
    data = data[:,np.newaxis,:,:] 
    #data = data[:,:,:,np.newaxis]
    
    # Makes values floats between 0 and 1 (gives better results for neural nets)
    data = data.astype('float32')
    data /= 255
    
    # Save the data as numpy file for faster loading
    np.save(os.getcwd()+"/data/"+setType+suffix+".npy", data)

    
### Labels preprocessing ###

# Load labels
y_train = pd.read_csv(os.getcwd()+"/data/trainLabels.csv").values[:,1] #Keep only label

# Convert labels to one-hot vectors
Y_train = np.zeros((y_train.shape[0], len(np.unique(y_train))))

for i in range(y_train.shape[0]):
    Y_train[i][label2int(y_train[i])] = 1 # One-hot

# Save preprocessed label to nupy file for faster loading
np.save(os.getcwd()+"/data/"+"labelsPreproc.npy", Y_train)

In [ ]:
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from sklearn.cross_validation import train_test_split
from keras.optimizers import SGD, Adam


batch_size = 400
nb_classes = 62 # A-Z, a-z and 0-9
nb_epoch = 100

# Input image dimensions
img_rows, img_cols = 48, 48

# Path of data files
path = "../data"

# Load the preprocessed data and labels
X_train_all = np.load(os.getcwd()+"/data/"+"/trainPreproc_"+str(img_rows)+"_"+str(img_cols)+".npy")
Y_train_all = np.load(os.getcwd()+"/data/"+"/labelsPreproc.npy")

# Do multiple learnings and predictions with the aim of averaging them
for runID in range (18):    
    # Split in train and validation sets to get the "best" model.
    X_train, X_val, Y_train, Y_val = \
        train_test_split(X_train_all, Y_train_all, test_size=0.25, stratify=np.argmax(Y_train_all, axis=1))
   
                             
    # Parametrize the image augmentation class
    datagen = ImageDataGenerator(
        rotation_range = 25,
        zca_whitening=True,
        width_shift_range = 0.15,
        height_shift_range = 0.15,
        shear_range = 0.4,
        zoom_range = 0.4, 
        horizontal_flip=True,
        fill_mode='nearest',
        channel_shift_range = 0.2,
        channel_flip = True, # You must modify the ImageDataGenerator class for that parameter to work
        channel_flip_max = 1.) # You must modify the ImageDataGenerator class for that parameter to work
    #datagen.fit(X_train)
    
    ### CNN MODEL ###
    model = Sequential()

    model.add(Convolution2D(128, 3, 3, border_mode='same', init='he_normal', activation = 'relu', input_shape=(1,img_rows, img_cols)))
    model.add(Convolution2D(128, 3, 3, border_mode='same', init='he_normal', activation = 'relu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Convolution2D(256, 3, 3, border_mode='same', init='he_normal', activation = 'relu'))
    model.add(Convolution2D(256, 3, 3, border_mode='same', init='he_normal', activation = 'relu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Convolution2D(512, 3, 3, border_mode='same', init='he_normal', activation = 'relu'))
    model.add(Convolution2D(512, 3, 3, border_mode='same', init='he_normal', activation = 'relu'))
    model.add(Convolution2D(512, 3, 3, border_mode='same', init='he_normal', activation = 'relu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(4096, init='he_normal', activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, init='he_normal', activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, init='he_normal', activation = 'softmax'))

    ### LEARNING ###

    # First, use AdaDelta for some epochs because AdaMax gets stuck
    lrate = 0.001
    decay = lrate/30
    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
    #adamax = Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='categorical_crossentropy', 
                  optimizer='AdaDelta',  
                  metrics=["accuracy"])
    

                  
    # 20 epochs is sufficient
    model.fit(X_train, Y_train, batch_size=batch_size,
                        nb_epoch=30, 
                        validation_data=(X_val, Y_val),
                        verbose=1)
                      
    # Now, use AdaMax
    model.compile(loss='categorical_crossentropy', 
                 optimizer='AdaMax',  
                 metrics=["accuracy"])

    # We want to keep the best model. This callback will store 
    # in a file the weights of the model with the highest validation accuracy  
    saveBestModel = ModelCheckpoint("best.kerasModelWeights64", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True)

    

In [ ]:
# Make the model learn using the image generator
    model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                        samples_per_epoch=len(X_train),
                        nb_epoch=nb_epoch, 
                        validation_data=(X_val, Y_val),
                        callbacks=[saveBestModel],
                        verbose=1)

    ### PREDICTION ###
                        
    # Load the model with the highest validation accuracy
    model.load_weights("best.kerasModelWeights")

    # Load Kaggle test set
    X_test = np.load(os.getcwd()+"/data/testPreproc_"+str(img_rows)+"_"+str(img_cols)+".npy")

    # Predict the class (give the index in the one-hot vector of the most probable class)
    Y_test_pred = model.predict_classes(X_test)
    
    # Translate integers to character labels
    vInt2label = np.vectorize(int2label)
    Y_test_pred = vInt2label(Y_test_pred)
    
    # Save the predicitions in Kaggle format
    np.savetxt(os.getcwd()+"/data/CNN_pred_"+str(runID)+".csv", np.c_[range(6284,len(Y_test_pred)+6284),Y_test_pred], delimiter=',', header = 'ID,Class', comments = '', fmt='%s')

In [ ]:
model.load_weights("best.kerasModelWeights")

    # Load Kaggle test set
X_test = np.load(os.getcwd()+"/data/testPreproc_"+str(img_rows)+"_"+str(img_cols)+".npy")

    # Predict the class (give the index in the one-hot vector of the most probable class)
Y_test_pred = model.predict_classes(X_test)
    
    # Translate integers to character labels
vInt2label = np.vectorize(int2label)
Y_test_pred = vInt2label(Y_test_pred)
    
    # Save the predicitions in Kaggle format
np.savetxt(os.getcwd()+"/data/CNN_pred_"+str(runID)+".csv", np.c_[range(6284,len(Y_test_pred)+6284),Y_test_pred], delimiter=',', header = 'ID,Class', comments = '', fmt='%s')

In [ ]:
import pandas as pd
import numpy as np
import glob

# Path of data files
path = "../data"


# List of prediction files in path
allPredsFiles = glob.glob(os.getcwd()+"/data/CNN_pred_*.csv")

# Array containg all the prediction
allPreds = []

for file in allPredsFiles:
    preds = pd.read_csv(file).values
    predictionTemplate = preds # Keep one prediction as template
    allPreds.append(preds[:,1:2]) #1:2 keeps dimension and removes label

# Stacks all the predictions (easier to treat this way)
predsStacked = np.hstack(allPreds) 

# Create the averaged result array, copying the template
predsAveraged = np.array(predictionTemplate)

# For each prediction, get the most frequent one
for i in range(predsStacked.shape[0]):
    (values,counts) = np.unique(predsStacked[i], return_counts=True) # Use count to find most frequent one
    ind=np.argmax(counts) # Index of the most frequent label
    predsAveraged[i,1] = values[ind]  # gets the most frequent label
    
# Save the averaged predicitions using Kaggle format
np.savetxt(os.getcwd()+"/data/avg_pred.csv", np.c_[predsAveraged], delimiter=',', header = 'ID,Class', comments = '', fmt='%s')